In [38]:
import random
import time
from kafka import KafkaProducer
import json
from datetime import datetime

actions = ["login", "logout", "purchase", "click"]
data = []

for user_id in range(1, 10):
    for _ in range(random.randint(1, 10)):
        action = random.choice(actions)
        timestamp = datetime.now().isoformat()
        data.append({"user_id": user_id, "action": action, "timestamp": timestamp})

with open('materials/hw_11/client_actions.json', 'w') as f:
    json.dump(data, f)

producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    acks='all',
    retries=5,
    linger_ms=10
)

with open('materials/hw_11/client_actions.json') as f:
    data = json.load(f)
    for record in data:
        future = producer.send('test_topic', record)

producer.flush()
producer.close()

In [39]:
from kafka import KafkaConsumer
from collections import Counter
import json

consumer = KafkaConsumer('test_topic',
                         bootstrap_servers='localhost:9092',
                         auto_offset_reset='latest',
                         enable_auto_commit=True,
                         group_id='test-group',
                         value_deserializer=lambda x: json.loads(x.decode('utf-8')),
                         consumer_timeout_ms = 10000)


In [40]:
action_counter = Counter()

for message in consumer:
    action = message.value['action']
    action_counter[(message.value['user_id'], action)] += 1

most_active_users = action_counter.most_common()

for (user_id, action), count in most_active_users:
    print(f'User ID: {user_id}, Action: {action}, Count: {count}')
    
consumer.close()

User ID: 4, Action: purchase, Count: 8
User ID: 9, Action: logout, Count: 8
User ID: 3, Action: login, Count: 7
User ID: 4, Action: login, Count: 7
User ID: 7, Action: purchase, Count: 7
User ID: 1, Action: purchase, Count: 6
User ID: 1, Action: click, Count: 6
User ID: 1, Action: logout, Count: 6
User ID: 5, Action: login, Count: 6
User ID: 7, Action: login, Count: 6
User ID: 1, Action: login, Count: 5
User ID: 2, Action: logout, Count: 5
User ID: 3, Action: purchase, Count: 5
User ID: 5, Action: click, Count: 5
User ID: 7, Action: click, Count: 5
User ID: 8, Action: logout, Count: 5
User ID: 3, Action: click, Count: 5
User ID: 5, Action: purchase, Count: 5
User ID: 2, Action: purchase, Count: 4
User ID: 3, Action: logout, Count: 4
User ID: 4, Action: logout, Count: 4
User ID: 4, Action: click, Count: 4
User ID: 8, Action: login, Count: 4
User ID: 6, Action: logout, Count: 3
User ID: 2, Action: click, Count: 2
User ID: 6, Action: login, Count: 2
User ID: 9, Action: click, Count: 2
Use